# Codelab 03: Feature Extraction and Object Classification

## Part 1: Load data

In [ ]:
from skimage.io import imread
import matplotlib.pyplot as plt
import numpy as np
from skimage.color import label2rgb

In [ ]:
# Define path
labels_filepath = r'../example_images/seed_labels.tif'
img_filepath = r'../example_images/seed_img.JPG'

# Read images
seed_rois = imread(labels_filepath)
img = imread(img_filepath)

# Split channel
img_ch0 = img[:,:,0]
img_ch1 = img[:,:,1]
img_ch2 = img[:,:,2]

# Define a range to visulize
x_start = 1000
x_end = 2000
y_start = 1000
y_end = 2000

# Display label on top of the image
seed_img_overlay = label2rgb(seed_rois, image=img_ch1, bg_label=0, alpha=0.3, bg_color=(1, 1, 1))
plt.figure(figsize=(4, 4))
plt.imshow(seed_img_overlay[x_start:x_end, y_start:y_end])

## Part 2: Measure object properties

In [ ]:
# Import regionprops_table from skimage.measure
from skimage.measure import ...

# Extract properties from the seed rois in the image channel 0
props_ch0 = ...
# Extract properties from the seed rois in the image channel 1
props_ch1 = ...

In [ ]:
# Import pandas
...

In [ ]:
# Convert the properties to pandas dataframe
props_ch0_df = ...
props_ch1_df = ...

In [ ]:
# Join the two dataframes
# lsuffix and rsuffix are used to differentiate the columns from the two dataframes
props_df = ...

In [ ]:
# Display the first few rows of the dataframe with *.head() method
...

In [ ]:
# Visulize the distribution of data points in color space

plt.figure(figsize=(4,4))
plt.plot(...
plt.xlabel('ch0 Intensity')
plt.ylabel('ch1 Intensity')

In [ ]:
# You can also use the seaborn library, a higher level library that utilize pandas and matplotlib to plot graphs

# “Seaborn is a library for making statistical graphics in Python. It builds on top of matplotlib and integrates 
# closely with pandas data structures.”

# Import seaborn
...

# Plot the joint distribution of the two channels
...

In [ ]:
# But we are also interested in looking at the distribution of data points in other dimensions. 
# We can use other features of a scatter plot to represent other aspects of the data
...

## Part 3: Dimensional Reduction

### Sec01: Dimension Reduction with PCA

In [ ]:
# Data scaling: ensuring that one feature doesn’t dominate just because of its scale.
# Import StandardScaler from sklearn
from sklearn.preprocessing import StandardScaler
# Create a StandardScaler object
std_scaler = ...
# Fit and transform the data
props_data = props_df.values
props_data_scaled = ...

In [ ]:
# Import PCA from sklearn
from sklearn.decomposition import PCA

# Create a PCA object with 2 components
...

# Fit and transform the scaled data
seed_pca = ...

In [ ]:
# Print the explained variance ratio
# Explined variance ratio is the ratio of the variance of the data points in the new space to the variance of the data points in the original space
print(f"Explained variance ratio: {...}")

In [ ]:
# Plot the data points in the new space
plt.figure(figsize=(4,4))
plt.scatter(...
plt.xlabel('PC1')
plt.ylabel('PC2')

### Sec02: Dimension Reduction with UMAP

In [ ]:
import umap.umap_ as umap
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# "UMAP is a stochastic algorithm – it makes use of randomness both to speed up approximation steps, 
# and to aid in solving hard optimization problems."
...

In [ ]:
# Create a UMAP object
reducer = ...

In [ ]:
# Fit and transform the scaled data
embedding = ...

In [ ]:
# Plot the data points in the new space
plt.scatter(...

## Part 4: Clustering/ Classification

### Sec01: K-mean

In [ ]:
# Import KMeans from sklearn
from sklearn.cluster import KMeans

In [ ]:
# Define the number of clusters
n_clusters = 4
kmeans = ...

In [ ]:
# Predict the cluster of each data point
kmeans_prediction = ...

In [ ]:
# Plot the data points in the new space with the cluster information
for i in range(n_clusters):
    plt.plot(...

### Sec02: HDBSCAN
How HDBSCAN works: https://hdbscan.readthedocs.io/en/latest/how_hdbscan_works.html

In [ ]:
# Import hdbscan
import hdbscan
# Create a HDBSCAN object, and fit the data
clusterer = ...
clusterer...

In [ ]:
# Predict the cluster of each data point
cluster_ids = np.unique(clusterer.labels_)
print(f'There are {len(cluster_ids)} clusters, with cluster ID: {cluster_ids}.')

In [ ]:
# Plot the data points in the new space with the cluster information
for cluster_id in cluster_ids:
    ...
plt.xlabel('UMAP1')
plt.ylabel('UMAP2')
plt.title('UMAP Analysis Results, labelled by HDBSCAN cluster ID')

## Part 5: Visulization of the clustering results

In [ ]:
props_df.shape[0]

In [ ]:


# Check the original image for the clustering result
num_roi = props_df.shape[0]

roi_cluster_labels = np.zeros_like(seed_rois)

for roi_id in range(num_roi):
    # Get ROI of a single seed
    seed_roi = ...

    # Add ROIs to the roi_cluster_labels, with label as cluster Id + 1 (+1 such that the label won't be background)
    roi_cluster_labels[seed_roi] = ...

In [ ]:
roi_cluster_labels_mask = np.ma.array(roi_cluster_labels, mask=roi_cluster_labels==0)
plt.figure(figsize = (10,10))
plt.subplot(121)
plt.imshow(img_ch1[y_start: y_end, x_start: x_end], interpolation='none', cmap='gray')
plt.subplot(122)
plt.imshow(roi_cluster_labels_mask[y_start: y_end, x_start: x_end], interpolation='none', cmap='prism')

In [ ]:
# First import module napari
import napari

# Create an empty viewer object
viewer = napari.Viewer()

# Use viewer.add_image() and pass the image as a variable to visulize the image. Similar to that for matplotlib, set options:
#  * colormap as 'gray'
#  * interpolation (interpolation2d) is 'nearest' (which correspond to minimum interpolation) by default, so no need to specify 
#  * name as 'Raw Image'
viewer.add_image(img_ch0, colormap = 'red', name= 'Raw Image', blending= 'additive')
viewer.add_image(img_ch1, colormap = 'green', name= 'Raw Image', blending= 'additive')
viewer.add_image(img_ch2, colormap = 'blue', name= 'Raw Image', blending= 'additive')

viewer.add_labels(roi_cluster_labels)

In [ ]:
viewer.add_labels(seed_rois)

## Compare the color distribution of groups

In [ ]:
# Add the cluster labels to the props_df
props_df.insert(...)

In [ ]:
props_df.head()

In [ ]:
sns.kdeplot(data=props_df, x="intensity_mean_ch0", hue="Cluster ID")

In [ ]:
sns.kdeplot(data=props_df, x="area", hue="Cluster ID")